In [1]:
import tensorflow as tf

# GPU를 사용할 수 있는지 확인
gpu_available = tf.config.list_physical_devices('GPU')

if gpu_available:
    print("GPU를 사용할 수 있습니다.")
    # GPU 디바이스 정보 출력
    for gpu in gpu_available:
        print("GPU 디바이스:", gpu)
else:
    print("GPU를 사용할 수 없습니다.")

GPU를 사용할 수 있습니다.
GPU 디바이스: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [29]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# cnn model vary kernel size
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

from tensorflow.keras.utils import to_categorical
import tensorflow.keras

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Reshape, LSTM, Conv1D, MaxPooling1D, Bidirectional, ConvLSTM1D
from tensorflow.keras.layers import Input, TimeDistributed, Conv1D, MaxPooling1D, BatchNormalization, GlobalAveragePooling1D, Activation
from tensorflow.keras.layers import Input, TimeDistributed, Conv1D, MaxPooling1D, BatchNormalization, Activation, UpSampling1D, Add, LayerNormalization
from tensorflow.keras.layers import add, multiply, GlobalAveragePooling1D
from keras import  backend as K
from tensorflow.python.keras.utils import np_utils
from sklearn.metrics import precision_score, recall_score, f1_score

from typing import Any, Dict, List, Tuple
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.api import VAR

In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns


def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, sep='\s+')
    return dataframe.values



# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = np.dstack(loaded)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'C:/Users/GACHON/Desktop/AiStudy/data/UCI HAR Dataset-20240320T080931Z-001/UCI HAR Dataset/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'C:/Users/GACHON/Desktop/AiStudy/data/UCI HAR Dataset-20240320T080931Z-001/UCI HAR Dataset/')

    #zero-offset class values
    trainy = trainy - 1
    testy = testy - 1

    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

    '''
    trainy_one_hot = to_categorical(trainy)
    testy_one_hot = to_categorical(testy)
    print(trainX.shape, trainy.shape, trainy_one_hot.shape, testX.shape, testy.shape, testy_one_hot.shape)
    return trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot
    '''

In [6]:
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 1) (2947, 128, 9) (2947, 1)


In [7]:
print("X train shape: ", trainX.shape)
print("Y train shape: ", trainy.shape)
print("X test shape: ", testX.shape)
print("Y test shape: ", testy.shape)

X train shape:  (7352, 128, 9)
Y train shape:  (7352, 1)
X test shape:  (2947, 128, 9)
Y test shape:  (2947, 1)


In [8]:
print("0: Walking:" ,np.where(trainy == 0)[0].size)
print("1: WU:" ,np.where(trainy == 1)[0].size)
print("2: WD:" ,np.where(trainy == 2)[0].size)
print("3: Sitting:" ,np.where(trainy == 3)[0].size)
print("4: Standing:" ,np.where(trainy == 4)[0].size)
print("5: Laying:" ,np.where(trainy == 5)[0].size)

0: Walking: 1226
1: WU: 1073
2: WD: 986
3: Sitting: 1286
4: Standing: 1374
5: Laying: 1407


In [9]:
unique, counts = np.unique(trainy, return_counts=True)
print ("Train data label statistics::")
print (np.asarray((unique, counts)).T)

unique, counts = np.unique(testy, return_counts=True)
print ("Test data label statistics::")
print (np.asarray((unique, counts)).T)

Train data label statistics::
[[   0 1226]
 [   1 1073]
 [   2  986]
 [   3 1286]
 [   4 1374]
 [   5 1407]]
Test data label statistics::
[[  0 496]
 [  1 471]
 [  2 420]
 [  3 491]
 [  4 532]
 [  5 537]]


In [10]:
# trainy에서 값이 0, 1, 2인 행의 인덱스 찾기
indices_to_remove = np.where((trainy == 0) | (trainy == 1) | (trainy == 2))[0]

# trainX와 trainy에서 해당 인덱스를 제거하여 새로운 trainX_filtered와 trainy_filtered 얻기
trainX_filtered = np.delete(trainX, indices_to_remove, axis=0)
trainy_filtered = np.delete(trainy, indices_to_remove, axis=0)

# trainX_filtered와 trainy_filtered의 shape 확인
print("Shape of trainX_filtered:", trainX_filtered.shape)
print("Shape of trainy_filtered:", trainy_filtered.shape)

Shape of trainX_filtered: (4067, 128, 9)
Shape of trainy_filtered: (4067, 1)


In [11]:
# trainy에서 값이 3인 것을 0으로, 값이 4인 것을 1로, 값이 5인 것을 2로 변경
trainy_filtered[trainy_filtered == 3] = 0
trainy_filtered[trainy_filtered == 4] = 1
trainy_filtered[trainy_filtered == 5] = 2

# 변경된 trainy_filtered의 값 확인
print("Modified trainy_filtered:\n", trainy_filtered)

Modified trainy_filtered:
 [[1]
 [1]
 [1]
 ...
 [2]
 [2]
 [2]]


In [12]:
# trainy에서 값이 0, 1, 2인 행의 인덱스 찾기
indices_to_remove = np.where((testy == 0) | (testy == 1) | (testy == 2))[0]

# trainX와 trainy에서 해당 인덱스를 제거하여 새로운 trainX_filtered와 trainy_filtered 얻기
testX_filtered = np.delete(testX, indices_to_remove, axis=0)
testy_filtered = np.delete(testy, indices_to_remove, axis=0)

# trainX_filtered와 trainy_filtered의 shape 확인
print("Shape of testX_filtered:", testX_filtered.shape)
print("Shape of testy_filtered:", testy_filtered.shape)

Shape of testX_filtered: (1560, 128, 9)
Shape of testy_filtered: (1560, 1)


In [13]:
# trainy에서 값이 3인 것을 0으로, 값이 4인 것을 1로, 값이 5인 것을 2로 변경
testy_filtered[testy_filtered == 3] = 0
testy_filtered[testy_filtered == 4] = 1
testy_filtered[testy_filtered == 5] = 2

# 변경된 trainy_filtered의 값 확인
print("Modified testy_filtered:\n", testy_filtered)

Modified testy_filtered:
 [[1]
 [1]
 [1]
 ...
 [2]
 [2]
 [2]]


In [14]:
trainy_one_hot = to_categorical(trainy_filtered)
testy_one_hot = to_categorical(testy_filtered)
print(trainy_filtered)
print(testy_filtered)

[[1]
 [1]
 [1]
 ...
 [2]
 [2]
 [2]]
[[1]
 [1]
 [1]
 ...
 [2]
 [2]
 [2]]


In [15]:
print("X train shape: ", trainX_filtered.shape)
print("Y train shape: ", trainy_filtered.shape)
print("Y train One hot shape: ", trainy_one_hot.shape)
print("X test shape: ", testX_filtered.shape)
print("Y test shape: ", testy_filtered.shape)
print("Y test One hot shape: ", testy_one_hot.shape)

X train shape:  (4067, 128, 9)
Y train shape:  (4067, 1)
Y train One hot shape:  (4067, 3)
X test shape:  (1560, 128, 9)
Y test shape:  (1560, 1)
Y test One hot shape:  (1560, 3)


In [16]:
print("0: Sitting:" ,np.where(trainy_filtered == 0)[0].size)
print("1: Standing:" ,np.where(trainy_filtered == 1)[0].size)
print("2: Laying:" ,np.where(trainy_filtered == 2)[0].size)

0: Sitting: 1286
1: Standing: 1374
2: Laying: 1407


In [17]:
unique, counts = np.unique(trainy_filtered, return_counts=True)
print ("Train data label statistics::")
print (np.asarray((unique, counts)).T)

unique, counts = np.unique(testy_filtered, return_counts=True)
print ("Test data label statistics::")
print (np.asarray((unique, counts)).T)

Train data label statistics::
[[   0 1286]
 [   1 1374]
 [   2 1407]]
Test data label statistics::
[[  0 491]
 [  1 532]
 [  2 537]]


In [18]:
X_train_all = trainX_filtered   # at this stage, the data includes both dynamic and static HAR data
y_train_all = trainy_filtered

X_test_all = testX_filtered
y_test_all = testy_filtered

In [19]:
from sklearn.preprocessing import StandardScaler

def scale_data(trainX_filtered, testX_filtered):
 # remove overlap
 cut = int(trainX_filtered.shape[1] / 2)
 longX = trainX_filtered[:, -cut:, :]
 # flatten windows
 longX = longX.reshape((longX.shape[0] * longX.shape[1], longX.shape[2]))
 # flatten train and test
 flatTrainX = trainX_filtered.reshape((trainX_filtered.shape[0] * trainX_filtered.shape[1], trainX_filtered.shape[2]))
 flatTestX = testX_filtered.reshape((testX_filtered.shape[0] * testX_filtered.shape[1], testX_filtered.shape[2]))

 s = StandardScaler()
 # fit on training data
 s.fit(longX)
 # apply to training and test data
 longX = s.transform(longX)
 flatTrainX = s.transform(flatTrainX)
 flatTestX = s.transform(flatTestX)
 # reshape
 flatTrainX = flatTrainX.reshape((trainX_filtered.shape))
 flatTestX = flatTestX.reshape((testX_filtered.shape))
 return flatTrainX, flatTestX

In [20]:
trainX, testX = scale_data(trainX_filtered, testX_filtered)

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train_one_hot,y_val_one_hot,y_train,y_val=train_test_split(trainX_filtered, trainy_one_hot, trainy_filtered,test_size=0.3,random_state=100)

In [22]:
n_timesteps, n_features, n_outputs = trainX_filtered.shape[1], trainX_filtered.shape[2], trainy_one_hot.shape[1]

In [23]:
n_status = len(np.unique(y_train_one_hot))
n_status

2

In [41]:
trainX = trainX.reshape(-1, trainX.shape[-1])
testX = testX.reshape(-1, testX.shape[-1])
X_val = X_val.reshape(-1, X_val.shape[-1])

In [42]:
var_model = VAR(trainX)
var_result = var_model.fit(maxlags=1)

var_forecast_steps = 10
var_forecast = var_result.forecast(trainX, steps=var_forecast_steps)

lstm_model = Sequential()
lstm_model.add(LSTM(100, input_shape=(n_timesteps, n_features), return_sequences = True))
lstm_model.add(LSTM(150))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(150, activation='relu'))
lstm_model.add(Dense(n_outputs, activation = 'softmax'))
lstm_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
lstm_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 128, 100)          44000     
                                                                 
 lstm_8 (LSTM)               (None, 150)               150600    
                                                                 
 dropout_5 (Dropout)         (None, 150)               0         
                                                                 
 dense_9 (Dense)             (None, 150)               22650     
                                                                 
 dense_10 (Dense)            (None, 3)                 453       
                                                                 
Total params: 217,703
Trainable params: 217,703
Non-trainable params: 0
_________________________________________________________________


In [43]:
# validation_split = 0.1
batch_size = 128

train_epochs = 20
history = lstm_model.fit(trainX, trainy_one_hot, epochs=train_epochs, batch_size=batch_size, verbose=True,  validation_data = (X_val, y_val_one_hot), shuffle=True)

ValueError: Data cardinality is ambiguous:
  x sizes: 520576
  y sizes: 4067
Make sure all arrays contain the same number of samples.